In [1163]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import torch.nn as nn
import torch.optim as optim

CVS_PATH = 'student/student-por.csv'

In [1164]:

df = pd.read_csv(CVS_PATH, delimiter=';')
print(df['romantic'])

0       no
1       no
2       no
3      yes
4       no
      ... 
644     no
645     no
646     no
647     no
648     no
Name: romantic, Length: 649, dtype: object


Now We will define Columns that are nominal (not ordinal) and need one-hot encoding

In [1165]:

COLUMNS_TO_CATEGORIZE = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid',
                         'activities', 'nursery', 'higher', 'internet', 'romantic']

In [1166]:
df = pd.get_dummies(
    df, 
    columns=COLUMNS_TO_CATEGORIZE,
    prefix=COLUMNS_TO_CATEGORIZE
).astype(int)

print(df['romantic_yes'])

0      0
1      0
2      0
3      1
4      0
      ..
644    0
645    0
646    0
647    0
648    0
Name: romantic_yes, Length: 649, dtype: int64


In [1167]:
COLUMNS_TO_NORMALIZE = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3'] 

In [1168]:
scaler = StandardScaler()
df[COLUMNS_TO_NORMALIZE] = scaler.fit_transform(df[COLUMNS_TO_NORMALIZE])

In [1169]:

print([c for c in df.columns if "roman" in c])
X = df.drop(columns=['G3', 'romantic_yes', 'romantic_no']).values.astype(np.float32)
y_grade = df['G3'].values.astype(np.float32).reshape(-1, 1)  
y_romantic = df['romantic_yes'].values.astype(int)

print(X)

['romantic_no', 'romantic_yes']
[[ 1.0316951   1.3102156   1.5407155  ...  1.          1.
   0.        ]
 [ 0.21013668 -1.3360394  -1.1888323  ...  1.          0.
   1.        ]
 [-1.4329803  -1.3360394  -1.1888323  ...  1.          0.
   1.        ]
 ...
 [ 1.0316951  -1.3360394  -1.1888323  ...  1.          1.
   0.        ]
 [ 0.21013668  0.42813063 -1.1888323  ...  1.          0.
   1.        ]
 [ 1.0316951   0.42813063 -0.27898306 ...  1.          0.
   1.        ]]


In [1170]:
X_trainval, X_test, y_grade_trainval, y_grade_test, y_rom_trainval, y_rom_test = train_test_split(
    X, y_grade, y_romantic, test_size=0.15, random_state=42
)
X_train, X_val, y_grade_train, y_grade_val, y_rom_train, y_rom_val = train_test_split(
    X_trainval, y_grade_trainval, y_rom_trainval, test_size=0.15, random_state=42
)

In [1171]:

class StudentDatasetPor(Dataset):
    def __init__(self, X, y_grade, y_romantic):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_grade = torch.tensor(y_grade, dtype=torch.float32)
        self.y_romantic = torch.tensor(y_romantic, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y_grade[idx], self.y_romantic[idx]

In [1172]:
train_dataset = StudentDatasetPor(X_train, y_grade_train, y_rom_train)
val_dataset = StudentDatasetPor(X_val, y_grade_val, y_rom_val)
test_dataset = StudentDatasetPor(X_test, y_grade_test, y_rom_test)

In [1173]:
BATCH_SIZE = 16
FIRST_NEURON_N = 64
SECOND_NEURON_N = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [1174]:
class StudentMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.shared = nn.Sequential(
            nn.Linear(input_dim, FIRST_NEURON_N),
            nn.BatchNorm1d(FIRST_NEURON_N),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(FIRST_NEURON_N, SECOND_NEURON_N),
            nn.BatchNorm1d(SECOND_NEURON_N),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        
        self.grade_head = nn.Linear(SECOND_NEURON_N, 1)
        
        self.romantic_head = nn.Linear(SECOND_NEURON_N, 2)

    def forward(self, x):
        features = self.shared(x)

        grade_pred = self.grade_head(features)
        romantic_logit = self.romantic_head(features)

        return grade_pred, romantic_logit

In [1175]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 1e-3

In [1176]:

model = StudentMLP(X.shape[1]).to(device)

class_counts = np.bincount(y_romantic)
class_weights = torch.FloatTensor([1.0/class_counts[0], 1.0/class_counts[1]]).to(device)

class_weights = class_weights / class_weights.mean()
criterion_grade = nn.MSELoss()               
criterion_romantic = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
EPOCHS = 50

In [1177]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x_batch, y_grade_batch, y_romantic_batch in train_loader:
        x_batch = x_batch.to(device)
        y_grade_batch = y_grade_batch.to(device)
        y_rom_batch = y_romantic_batch.to(device)

        optimizer.zero_grad()

        grade_pred, romantic_logits = model(x_batch)

        loss_grade = criterion_grade(grade_pred, y_grade_batch)
        loss_romantic = criterion_romantic(romantic_logits, y_romantic_batch)

        loss = loss_grade + loss_romantic

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x_batch.size(0)
    
    total_loss /= len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {total_loss:.4f}")

Epoch [1/50], Train Loss: 1.8849
Epoch [2/50], Train Loss: 1.2673
Epoch [3/50], Train Loss: 1.1398
Epoch [4/50], Train Loss: 1.0279
Epoch [5/50], Train Loss: 1.0195
Epoch [6/50], Train Loss: 0.9759
Epoch [7/50], Train Loss: 0.9426
Epoch [8/50], Train Loss: 0.9474
Epoch [9/50], Train Loss: 0.8914
Epoch [10/50], Train Loss: 0.8939
Epoch [11/50], Train Loss: 0.9020
Epoch [12/50], Train Loss: 0.9008
Epoch [13/50], Train Loss: 0.8432
Epoch [14/50], Train Loss: 0.8654
Epoch [15/50], Train Loss: 0.9052
Epoch [16/50], Train Loss: 0.8532
Epoch [17/50], Train Loss: 0.7958
Epoch [18/50], Train Loss: 0.8227
Epoch [19/50], Train Loss: 0.8486
Epoch [20/50], Train Loss: 0.8196
Epoch [21/50], Train Loss: 0.8271
Epoch [22/50], Train Loss: 0.7775
Epoch [23/50], Train Loss: 0.7955
Epoch [24/50], Train Loss: 0.7774
Epoch [25/50], Train Loss: 0.7602
Epoch [26/50], Train Loss: 0.7514
Epoch [27/50], Train Loss: 0.7772
Epoch [28/50], Train Loss: 0.7828
Epoch [29/50], Train Loss: 0.7314
Epoch [30/50], Train Lo

In [1178]:
def evaluate_model(model, data_loader, device):
    model.eval()
    
    total_loss = 0
    total_loss_grade = 0
    total_loss_romantic = 0
    
    with torch.no_grad():
        for x_batch, y_grade_batch, y_rom_batch in data_loader:
            x_batch = x_batch.to(device)
            y_grade_batch = y_grade_batch.to(device)
            y_rom_batch = y_rom_batch.to(device)
            
            grade_pred, romantic_logits = model(x_batch)
            
            loss_grade = criterion_grade(grade_pred, y_grade_batch)
            loss_romantic = criterion_romantic(romantic_logits, y_rom_batch)
            loss = loss_grade + loss_romantic
            
            total_loss += loss.item() * x_batch.size(0)
            total_loss_grade += loss_grade.item() * x_batch.size(0)
            total_loss_romantic += loss_romantic.item() * x_batch.size(0)
    
    # Calculate metrics
    avg_loss = total_loss / len(data_loader.dataset)
    avg_loss_grade = total_loss_grade / len(data_loader.dataset)
    avg_loss_romantic = total_loss_romantic / len(data_loader.dataset)
    
    return {
        'total_loss': avg_loss,
        'grade_loss': avg_loss_grade,
        'romantic_loss': avg_loss_romantic,
    }

In [1179]:
print(evaluate_model(model=model, data_loader=val_loader, device=device))

{'total_loss': 1.0301241242741963, 'grade_loss': 0.1882410246026085, 'romantic_loss': 0.8418830949139883}


In [ ]:
def evaluate_test(model, test_loader, device):
    model.eval()

    all_grade_preds = []
    all_grade_true = []

    all_romantic_preds = []
    all_romantic_true = []

    with torch.no_grad():
        for x_batch, y_grade_batch, y_romantic_batch in test_loader:
            x_batch = x_batch.to(device)
            y_grade_batch = y_grade_batch.to(device)
            y_romantic_batch = y_romantic_batch.to(device)

            grade_out, romantic_out = model(x_batch)

            all_grade_preds.extend(grade_out.cpu().numpy().flatten())
            all_grade_true.extend(y_grade_batch.cpu().numpy().flatten())

            romantic_pred_labels = romantic_out.argmax(dim=1)
            all_romantic_preds.extend(romantic_pred_labels.cpu().numpy())
            all_romantic_true.extend(y_romantic_batch.cpu().numpy())


    mae = mean_absolute_error(all_grade_true, all_grade_preds)

    accuracy = accuracy_score(all_romantic_true, all_romantic_preds)

    f1_yes = f1_score(all_romantic_true, all_romantic_preds, pos_label=1)

    return {
        "grade_MAE": mae,
        "romantic_accuracy": accuracy,
        "romantic_f1_yes": f1_yes
    }



In [1181]:
print(evaluate_test(model=model, test_loader=test_loader, device=device))

{'grade_MAE': 0.2845036464420204, 'romantic_accuracy': 0.5408163265306123, 'romantic_f1_yes': 0.3835616438356164}
